In [118]:
#!touch 0-l2_reg_cost.py
!chmod +x *.py
#!cp 0-l2_reg_cost.py 2-l2_reg_cost.py

In [103]:
#!/usr/bin/env python3

import numpy as np
#l2_reg_cost = __import__('0-l2_reg_cost').l2_reg_cost np.linalg.norm
def l2_reg_cost(cost, lambtha, weights, L, m):
    """ doc """
    sw = 0
    for i in range(1, L+1):
        sw += np.linalg.norm(weights['W'+str(i)], ord='fro')
    l2 = (lambtha*sw)/(2*m)
    return (cost+l2)

if __name__ == '__main__':
    np.random.seed(0)

    weights = {}
    weights['W1'] = np.random.randn(256, 784)
    weights['W2'] = np.random.randn(128, 256)
    weights['W3'] = np.random.randn(10, 128)

    cost = np.abs(np.random.randn(1))

    print(cost)
    cost = l2_reg_cost(cost, 0.1, weights, 3, 1000)
    print(cost)

[0.41842822]
[0.45158952]


In [117]:
#!/usr/bin/env python3

import numpy as np
#l2_reg_gradient_descent = __import__('1-l2_reg_gradient_descent').l2_reg_gradient_descent

def l2_reg_gradient_descent(Y, weights, cache, alpha, lambtha, L):
    """ doc """
    m = Y.shape[1]

    def dw(dz, x):
        """ weight derivative """
        return np.matmul(dz, x.T)/m

    def db(dz):
        """ bias derivative"""
        return np.mean(dz, axis=1, keepdims=True)

    def der(x):
        """ tanh derivative """
        return 1 - (x**2)

    def dz(wi, dzi, gprimei):
        """ z derivative """
        x = np.matmul(wi.T, dzi)
        return np.multiply(gprimei, x)

    n = L
    wb = weights.copy()
    dzi = np.subtract(cache['A'+str(n)], Y)
    for i in reversed(range(1, n+1)):
        Ai = cache['A'+str(i)]
        Ai_1 = cache['A'+str(i-1)]
        b = wb['b'+str(i)]
        if i == n:
            dzi = np.subtract(cache['A'+str(n)], Y)
        else:
            w = wb['W'+str(i+1)]
            dzi = dz(w, dzi, der(Ai))
        dwi = dw(dzi, Ai_1)
        dbi = db(dzi)
        weights['b'+str(i)] = wb['b'+str(i)]*(1-alpha*lambtha/m) - dbi
        weights['W'+str(i)] = wb['W'+str(i)]*(1-alpha*lambtha/m) - dwi
        """weights['b'+str(i)] = wb['b'+str(i)]-alpha*(dbi + lambtha/m*wb['b'+str(i)])
        weights['W'+str(i)] = wb['W'+str(i)]-alpha*(dwi + lambtha/m*wb['W'+str(i)])"""

def one_hot(Y, classes):
    """convert an array to a one-hot matrix"""
    m = Y.shape[0]
    one_hot = np.zeros((classes, m))
    one_hot[Y, np.arange(m)] = 1
    return one_hot

if __name__ == '__main__':
    lib= np.load('../data/MNIST.npz')
    X_train_3D = lib['X_train']
    Y_train = lib['Y_train']
    X_train = X_train_3D.reshape((X_train_3D.shape[0], -1)).T
    Y_train_oh = one_hot(Y_train, 10)

    np.random.seed(0)

    weights = {}
    weights['W1'] = np.random.randn(256, 784)
    weights['b1'] = np.zeros((256, 1))
    weights['W2'] = np.random.randn(128, 256)
    weights['b2'] = np.zeros((128, 1))
    weights['W3'] = np.random.randn(10, 128)
    weights['b3'] = np.zeros((10, 1))

    cache = {}
    cache['A0'] = X_train
    cache['A1'] = np.tanh(np.matmul(weights['W1'], cache['A0']) + weights['b1'])
    cache['A2'] = np.tanh(np.matmul(weights['W2'], cache['A1']) + weights['b2'])
    Z3 = np.matmul(weights['W3'], cache['A2']) + weights['b3']
    cache['A3'] = np.exp(Z3) / np.sum(np.exp(Z3), axis=0)
    print(weights['W1'])
    print("*****************")
    l2_reg_gradient_descent(Y_train_oh, weights, cache, 0.1, 0.1, 3)
    print(weights['W1'])

[[ 1.76405235  0.40015721  0.97873798 ...  0.52130375  0.61192719
  -1.34149673]
 [ 0.47689837  0.14844958  0.52904524 ...  0.0960042  -0.0451133
   0.07912172]
 [ 0.85053068 -0.83912419 -1.01177408 ... -0.07223876  0.31112445
  -1.07836109]
 ...
 [-0.60467085  0.54751161 -1.23317415 ...  0.82895532  1.44161136
   0.18972404]
 [-0.41044606  0.85719512  0.71789835 ... -0.73954771  0.5074628
   1.23022874]
 [ 0.43129249  0.60767018 -0.07749988 ... -0.26611561  2.52287972
   0.73131543]]
*****************
[[ 1.76405199  0.40015713  0.97873779 ...  0.52130364  0.61192707
  -1.34149646]
 [ 0.47689827  0.14844955  0.52904513 ...  0.09600419 -0.04511329
   0.07912171]
 [ 0.85053051 -0.83912402 -1.01177388 ... -0.07223874  0.31112438
  -1.07836088]
 ...
 [-0.60467073  0.5475115  -1.2331739  ...  0.82895516  1.44161107
   0.189724  ]
 [-0.41044598  0.85719495  0.71789821 ... -0.73954756  0.5074627
   1.2302285 ]
 [ 0.4312924   0.60767006 -0.07749987 ... -0.26611556  2.52287922
   0.73131529]]


In [131]:
#!/usr/bin/env python3

import numpy as np
import tensorflow as tf
#l2_reg_cost = __import__('2-l2_reg_cost').l2_reg_cost
def l2_reg_cost(cost):
    """ doc """
    
    return tf.contrib.losses.compute_weighted_loss(cost)

def one_hot(Y, classes):
    """convert an array to a one-hot matrix"""
    m = Y.shape[0]
    oh = np.zeros((classes, m))
    oh[Y, np.arange(m)] = 1
    return oh

np.random.seed(4)
m = np.random.randint(1000, 2000)
c = 10
lib= np.load('../data/MNIST.npz')

X = lib['X_train'][:m].reshape((m, -1))
Y = one_hot(lib['Y_train'][:m], c).T

n0 = X.shape[1]
n1, n2 = np.random.randint(10, 1000, 2)

lam = np.random.uniform(0.01)
tf.set_random_seed(0)

x = tf.placeholder(tf.float32, (None, n0))
y = tf.placeholder(tf.float32, (None, c))

a1 = tf.layers.Dense(n1, activation=tf.nn.tanh, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(mode="FAN_AVG"), kernel_regularizer=tf.contrib.layers.l2_regularizer(lam))(x)
a2 = tf.layers.Dense(n2, activation=tf.nn.sigmoid, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(mode="FAN_AVG"), kernel_regularizer=tf.contrib.layers.l2_regularizer(lam))(a1)
y_pred = tf.layers.Dense(c, activation=None, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(mode="FAN_AVG"), kernel_regularizer=tf.contrib.layers.l2_regularizer(lam))(a2)

cost = tf.losses.softmax_cross_entropy(y, y_pred)

l2_cost = l2_reg_cost(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(l2_cost, feed_dict={x: X, y: Y}))


Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.


InvalidArgumentError: Expected begin[0] == 0 (got 0) and size[0] == 0 (got 1) when input.dim_size(0) == 0
	 [[node weighted_loss/Slice (defined at /home/khouloud/.local/lib/python3.6/site-packages/tensorflow/contrib/losses/python/losses/loss_ops.py:175)  = Slice[Index=DT_INT32, T=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](weighted_loss/Sum/reduction_indices, softmax_cross_entropy_loss_12/Const, softmax_cross_entropy_loss_12/xentropy/Slice/begin)]]

Caused by op 'weighted_loss/Slice', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/khouloud/.local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/khouloud/.local/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/khoulou